In [33]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import plotly
import plotly.graph_objs as go
import warnings
import os
import plotly.io as pio

warnings.filterwarnings('ignore')

In [11]:
def convert_to_binary(row):
    if row < 9.5:
        return 0
    elif row >= 9.5:
        return 1
    else:
        return np.nan

#### Data

In [20]:
main = pd.read_csv('./training_test_data/train.csv')

In [21]:
sfa_owner_df = pd.read_excel('./feature_dictionary/feature_dictionary_final.xlsx')

In [22]:
sfa_owner_df.head()

,ID,Variable_Name,Description,CONT_F
0,1,HOME_LG_GOALS_L1W,Home team Number of goals scored 1 week ago in...,cont
1,2,HOME_LG_GOALS_L3W_COMB,Home team Number of goals scored in last 3 wee...,cont
2,3,HOME_LG_GOALS_H_L3W,Home team Number of goals scored 3 week ago in...,cont
3,4,HOME_LG_GOALS_A_L1W,Home team Number of goals scored 1 week ago in...,cont
4,5,HOME_LG_GOALS_A_L3W_COMB,Home team Number of goals scored in last 3 wee...,cont


In [23]:
cont_features = list(sfa_owner_df[sfa_owner_df['CONT_F']=='cont']['Variable_Name'])
cat_features = list(sfa_owner_df[sfa_owner_df['CONT_F']=='cat']['Variable_Name'])

all_features = list(sfa_owner_df['Variable_Name'])

In [24]:
main.rename({'Unnamed: 0':'TALEP_NO'},axis='columns',inplace=True)

In [25]:
main['DATASET']='MD'

In [26]:
main['TARGET2']=main['TARGET'].copy().apply(convert_to_binary)
main = main[['TALEP_NO','TARGET2','DATASET']+all_features]

## SFA

In [35]:
exec(open('./single_factor_analysis/sfa_code.py').read())

In [37]:
c_values = []

for feat in all_features:
    graph_stats, stats_df, minimum, maximum = sfa_cont(main, feat, 'TARGET2', 50)

    exec(open('./single_factor_analysis/draw_table.py').read())
    pio.write_image(fig0, './single_factor_analysis/images/table/'+feat+'_table.png')

    exec(open('./single_factor_analysis/draw_graph.py').read())
    pio.write_image(fig, './single_factor_analysis/images/graph/'+feat+'_graph.png')

    c_values.append([feat,stats_df.iloc[0]['org c-value']])

    print ('OK for '+feat)

Number of not-null observations: 1520
OK for HOME_LG_GOALS_L1W
Number of not-null observations: 1520
OK for HOME_LG_GOALS_L3W_COMB
Number of not-null observations: 1520
OK for HOME_LG_GOALS_H_L3W
Number of not-null observations: 1520
OK for HOME_LG_GOALS_A_L1W
Number of not-null observations: 1520
OK for HOME_LG_GOALS_A_L3W_COMB
Number of not-null observations: 1520
OK for HOME_LG_SHOTS_L3W
Number of not-null observations: 1520
OK for HOME_LG_SHOTS_H_L2W
Number of not-null observations: 1520
OK for HOME_LG_SHOTS_H_L3W_COMB
Number of not-null observations: 1520
OK for HOME_LG_SHOTS_A_L2W
Number of not-null observations: 1520
OK for HOME_LG_SHOTS_A_L3W_COMB
Number of not-null observations: 1520
OK for AWAY_LG_GOALS_L3W
Number of not-null observations: 1520
OK for AWAY_LG_GOALS_H_L2W
Number of not-null observations: 1520
OK for AWAY_LG_GOALS_H_L3W_COMB
Number of not-null observations: 1520
OK for AWAY_LG_GOALS_A_L2W
Number of not-null observations: 1520
OK for AWAY_LG_GOALS_A_L3W_COMB
Num

In [40]:
c_values_df = pd.DataFrame(c_values, columns=['feature','c_value'])

In [43]:
c_values_df.sort_values(by='c_value',ascending=False).to_csv('./feature_selection/feature_ranking.csv')